# ЧАСТЬ 1: PANDAS

In [ ]:
import pandas as pd

In [ ]:
movies=pd.read_csv('movies.csv',index_col='№')


In [ ]:
movies.head()

,Name,Release Date,Release Year,Review Link,Unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
№,,,,,,,,,,,,,,,,,,,,,,,
1.0,Toy Story (1995),01-Jan-1995,1995.0,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2.0,GoldenEye (1995),01-Jan-1995,1995.0,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3.0,Four Rooms (1995),01-Jan-1995,1995.0,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4.0,Get Shorty (1995),01-Jan-1995,1995.0,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5.0,Copycat (1995),01-Jan-1995,1995.0,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [ ]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

In [ ]:
ratings['User ID'].value_counts().head(1)

405    737
Name: User ID, dtype: int64

Пользователь №405 оставил больше всего отзывов (737 шт)

In [ ]:
sample = ratings.loc[ratings['User ID'] == 405]
sample.head()

,User ID,Movie ID,Rating,Timestamp
12276,405,56,4,885544911
12383,405,592,1,885548670
12430,405,1582,1,885548670
12449,405,171,1,885549544
12460,405,580,1,885547447


In [ ]:
#Add column with the sum of ratings (by all users) for each movie
sample = sample.merge(ratings.groupby("Movie ID", as_index=False).sum()[['Movie ID','Rating']].rename(columns={"Rating": "Sum of Ratings"}), how='inner', on="Movie ID")
#Add column with the quantity of ratings (by all users) for each movie
sample = sample.merge(ratings.groupby("Movie ID", as_index=False).count()[['Movie ID','Rating']].rename(columns={"Rating": "Quantity of Ratings"}), how='inner', on="Movie ID")
#Add columns with movie data (Movie name, release year, genres)
sample = sample.merge(movies.drop(columns=movies[["Release Date","Review Link"]]), how='inner', left_on="Movie ID", right_on="№")
sample.head()

,User ID,Movie ID,Rating,Timestamp,Sum of Ratings,Quantity of Ratings,Name,Release Year,Unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,405,56,4,885544911,1600,394,Pulp Fiction (1994),1994.0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
1,405,592,1,885548670,30,9,True Crime (1995),1995.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,405,1582,1,885548670,1,1,T-Men (1947),1947.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,405,171,1,885549544,252,65,Delicatessen (1991),1991.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
4,405,580,1,885547447,108,32,"Englishman Who Went Up a Hill, But Came Down a...",1995.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X, y = sample[['Sum of Ratings', 'Quantity of Ratings', 'Release Year', 'Unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']], sample['Movie ID']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [ ]:
X_train.describe


<bound method NDFrame.describe of      Sum of Ratings  Quantity of Ratings  Release Year  ...  Thriller  War  Western
426             715                  176        1993.0  ...         0    0        0
425             117                   43        1977.0  ...         0    0        0
361              42                   13        1987.0  ...         0    0        0
693              14                    5        1996.0  ...         0    1        0
63               23                    7        1995.0  ...         0    0        0
..              ...                  ...           ...  ...       ...  ...      ...
538             281                   82        1980.0  ...         0    0        0
638               1                    1        1988.0  ...         0    0        0
192             559                  154        1995.0  ...         1    1        0
549               7                    3        1986.0  ...         0    0        0
671             320                   86  

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()


In [ ]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
model.predict(X_train)

array([ 5.44116442e+02,  6.67907539e+02,  1.01917302e+03,  1.07527359e+03,
        9.63695130e+02,  5.42154875e+02,  1.10314564e+03,  6.10385020e+02,
        7.84076268e+02, -1.51490387e+03,  1.08707486e+03,  1.00792364e+03,
        1.08270374e+03,  8.31146999e+02,  7.92629792e+01,  2.39266287e+01,
       -2.02651776e+02,  5.13312553e+02,  6.35661996e+02,  6.04113080e+02,
        4.62672661e+02,  3.37666242e+02,  1.09530839e+03, -1.14426039e+01,
        7.26406563e+02,  1.07584134e+03,  8.63520755e+02,  5.39231574e+02,
        5.67080164e+02,  5.37489577e+02,  7.92099397e+02,  7.32040841e+02,
        1.07768627e+03,  7.82468196e+02,  7.69781193e+01,  1.08893134e+03,
        9.95676552e+02, -6.66162036e+02,  1.06588258e+03,  1.19231881e+03,
        3.67478326e+02,  1.16438317e+03,  4.30193924e+02,  2.14937689e+02,
        9.12109514e+02,  1.09992299e+03,  9.99582021e+02,  5.96369989e+02,
        4.11646614e+02,  6.88662393e+02,  1.10314564e+03,  4.95035821e+02,
        8.60969403e+02,  

In [ ]:
from sklearn.metrics import mean_squared_error


In [ ]:
mean_squared_error(y_test, model.predict(X_test))

112776.47305966716

# ЧАСТЬ 2: PySpark



## Setup

In [ ]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,426 k

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q mirror.linux-ia64.org/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!tar -xvf spark-3.1.2-bin-hadoop2.7.tgz

spark-3.1.2-bin-hadoop2.7/
spark-3.1.2-bin-hadoop2.7/R/
spark-3.1.2-bin-hadoop2.7/R/lib/
spark-3.1.2-bin-hadoop2.7/R/lib/sparkr.zip
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/worker/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/worker/worker.R
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/worker/daemon.R
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/tests/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/tests/testthat/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/profile/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/profile/shell.R
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/profile/general.R
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/R/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/R/SparkR
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/R/SparkR.rdx
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/R/SparkR.rdb
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/Meta/
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/Meta/features.rds
spark-3.1.2-bin-hadoop2.7/R/lib/SparkR/Meta/pac

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

## Diagrams

In [ ]:
spark_movies = spark.read.csv("movies.csv",inferSchema=True,header=True)

In [ ]:
spark_ratings = spark.read.csv("ratings.csv",inferSchema=True,header=True)

In [ ]:
spark_ratings.show()

+-------+--------+------+---------+
|User ID|Movie ID|Rating|Timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
|    298|     474|     4|884182806|
|    115|     265|     2|881171488|
|    253|     465|     5|891628467|
|    305|     451|     3|886324817|
|      6|      86|     3|883603013|
|     62|     257|     2|879372434|
|    286|    1014|     5|879781125|
|    200|     222|     5|876042340|
|    210|      40|     3|891035994|
|    224|      29|     3|888104457|
|    303|     785|     3|879485318|
|    122|     387|     5|879270459|
|    194|     274|     2|879539794|
|    291|    1042|     4|874834944|
|    234|    1184|     2|892079237|
+-------+--------+------+---------+
only showing top 20 rows



In [ ]:
avr_rating=spark_ratings.groupBy ('Movie ID'). avg ()[['Movie ID','avg(Rating)']] 

In [ ]:
spark_movies=spark_movies.withColumnRenamed('№', 'Movie ID') #переименуем колонку для соответствия

In [ ]:
rated_movies = avr_rating.join(spark_movies,['Movie ID'],how='inner')

rated_movies[['Movie ID', 'avg(Rating)', 'Name']].show ()

rated_movies=rated_movies.withColumnRenamed('avg(Rating)','Rating')

+--------+------------------+--------------------+
|Movie ID|       avg(Rating)|                Name|
+--------+------------------+--------------------+
|     496| 4.121212121212121|It's a Wonderful ...|
|     471|3.6108597285067874|Courage Under Fir...|
|     463| 3.859154929577465|Secret of Roan In...|
|     148|          3.203125|Ghost and the Dar...|
|    1342|               2.5|Convent, The (Con...|
|     833| 3.204081632653061|  Bulletproof (1996)|
|    1088| 2.230769230769231|  Double Team (1997)|
|    1591|3.1666666666666665|Duoluo tianshi (1...|
|    1238|             3.125|   Full Speed (1996)|
|    1580|               1.0|     Liebelei (1933)|
|    1645|               4.0|Butcher Boy, The ...|
|     392|3.5441176470588234|Man Without a Fac...|
|     623| 2.923076923076923|Angels in the Out...|
|     540| 2.511627906976744|  Money Train (1995)|
|     858|               1.0|Amityville: Dollh...|
|     737| 2.983050847457627|       Sirens (1994)|
|     243|2.4393939393939394|Ju

In [ ]:
Unknown_rating = rated_movies.groupBy ('Unknown')


In [ ]:
Unknown_rating_avg = Unknown_rating.avg('Rating').filter(Unknown_rating.avg('Rating')['Unknown'] == 1).show() #считаем среднюю оценку для жанра Unknown

+-------+------------------+
|Unknown|       avg(Rating)|
+-------+------------------+
|      1|2.2222222222222223|
+-------+------------------+



In [ ]:
best_worst=spark_ratings.groupBy ('Movie ID').count()

In [ ]:
best_worst=best_worst.join(spark_movies, how='inner', on="Movie ID")

In [ ]:
best_worst.orderBy('count', ascending=False)[['count','Name']]. show(5)#5 лучших фильмов

+-----+--------------------+
|count|                Name|
+-----+--------------------+
|  583|    Star Wars (1977)|
|  509|      Contact (1997)|
|  508|        Fargo (1996)|
|  507|Return of the Jed...|
|  485|    Liar Liar (1997)|
+-----+--------------------+
only showing top 5 rows



In [ ]:
best_worst.orderBy('count', ascending=True)[['count','Name']]. show(5)#5 худших фильмов

+-----+--------------------+
|count|                Name|
+-----+--------------------+
|    1|Stefano Quantesto...|
|    1|    Sleepover (1995)|
|    1|Three Lives and O...|
|    1|Butcher Boy, The ...|
|    1|     Liebelei (1933)|
+-----+--------------------+
only showing top 5 rows

